In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

NiN串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络

In [2]:
import time
import torch
from torch import nn,optim

import sys

In [4]:
sys.path.append("..")

In [5]:
import chapter_3.d2l as d2l

importing Jupyter notebook from ../chapter_3/d2l.ipynb


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
def nin_block(in_channels,out_channels,kernel_size,stride,padding):
    blk = nn.Sequential(nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding),
                        nn.ReLU(), 
                        nn.Conv2d(out_channels,out_channels,kernel_size=1), 
                        nn.ReLU(), 
                        nn.Conv2d(out_channels,out_channels,kernel_size=1),
                        nn.ReLU()
                       )
    return blk

In [13]:
import torch.nn.functional as F

In [14]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d,self).__init__()
    def forward(self,x):
        return F.avg_pool2d(x,kernel_size=x.size()[2:])

In [16]:
net = nn.Sequential(nin_block(1,96,kernel_size=11,stride=4,padding=0),
                   nn.MaxPool2d(kernel_size=3,stride=2),
                   nin_block(96,256,kernel_size=5,stride=1,padding=2),
                   nn.MaxPool2d(kernel_size=3,stride=2), 
                   nin_block(256,384,kernel_size=3,stride=1,padding=1), 
                   nn.MaxPool2d(kernel_size=3,stride=2),
                   nn.Dropout(0.5),
                   nin_block(384,10,kernel_size=3,stride=1,padding=1),
                   GlobalAvgPool2d(),
                   d2l.FlattenLayer())

In [17]:
X = torch.rand(1,1,224,224)

In [19]:
for name, blk in net.named_children():
    X = blk(X)
    print(name,'output shape: ',X.shape)

0 output shape:  torch.Size([1, 96, 54, 54])
1 output shape:  torch.Size([1, 96, 26, 26])
2 output shape:  torch.Size([1, 256, 26, 26])
3 output shape:  torch.Size([1, 256, 12, 12])
4 output shape:  torch.Size([1, 384, 12, 12])
5 output shape:  torch.Size([1, 384, 5, 5])
6 output shape:  torch.Size([1, 384, 5, 5])
7 output shape:  torch.Size([1, 10, 5, 5])
8 output shape:  torch.Size([1, 10, 1, 1])
9 output shape:  torch.Size([1, 10])


In [20]:
batch_size = 128

In [21]:
train_iter , test_iter = d2l.load_data_from_fmnist(batch_size,resize=224)

In [22]:
lr , num_epochs = 0.002, 5

In [23]:
optimizer = optim.Adam(net.parameters(),lr=lr)

In [ ]:
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cpu


* NiN重复使用由卷积层和代替全连接层的1×11×1卷积层构成的NiN块来构建深层网络。
* NiN去除了容易造成过拟合的全连接输出层，而是将其替换成输出通道数等于标签类别数的NiN块和全局平均池化层。
* NiN的以上设计思想影响了后面一系列卷积神经网络的设计。